# Import Widget Libraries

In [1]:
%load_ext autoreload   
%autoreload 2
import kyurem
from kyurem.core.idom_loader import reload_bundle
from kyurem.core.widget import WidgetModel

kyurem_client: 0.0.0


# Import Relevant Data

## Import Graph

In [2]:
from kyurem import Service
service = Service(kh = 'indeed.v0.0.1:612e3403')

In [3]:
schema = service.get_kh_edge_list()

## Import Corpus Data

In [5]:
import pandas as pd
df = pd.read_csv("indeed.v0.0.1_612e3403_sampled.csv")
data = df.to_dict('records')

In [6]:
service.load_corpus(data, 'parent_title', 'context', 'child_title')

# ESE Seed Creation

## Exploration

In [18]:
from kyurem import ExplorerESE


def init(): 
    # Fetch initial view data
    return {
        "subgraph": None,
        "children": service.get_children_node_distributions(),
        "relations": service.get_relation_distribution(),
        "datatable": None,
        "highlight": None,
    }

def focus(node, panel):
    # node can be in one of two formats:
    # node : { "node_label": str, "node_property": str, "node_property_value": str }
    # node : { "node_label": str, "title": str, "uuid": str }
    
    # Since the service functions use the first format, node must first
    # be converted to the proper format
    if "title" in node:
        node = {
            "node_label": node["node_label"],
            "node_property": "title",
            "node_property_value": node["title"]
        }
        
    # TODO: Return nodes in a consistent format from the service
    #       functions to streamline the above
    
    
    # Fetch the neighborhood around the input node 
    result = service.get_node_neighborhood(node)
    datatable = service.get_annotated_corpus(node)
    
    data = { "subgraph": None }
    if panel != "schema":
        data["schema"] = result["schema"]
    if panel != "children":
        data["children"] = service.get_children_node_distributions(node)

    # Use a list comprehension to convert the return format of the relation_dist 
    # to a bar-chart format
    data["relations"] = [
        {"x": relation["label"], "y": relation["count"], "type": type}
        for type, relations in result["relation_dist"].items()
        for relation in relations
    ]
    # TODO: Move conversion code into service
    if datatable["rows"]:
        data["datatable"] = datatable["rows"]
        data["highlight"] = datatable["highlight"]
    else:
        data["datatable"] = None
        data["highlight"] = None
    return data
  
# Create widget 
reload_bundle()
explorer = ExplorerESE({
        "init": init,
        "focus": focus
    }, schema)

explorer.show()

component(7f04946e0e20, self=<kyurem.widgets.StatefulWidgetBase.StatefulWidgetBase object at 0x7f0497abd9d0>)

LayoutWidget(Layout(component(7f04946e0e20, self=<kyurem.widgets.StatefulWidgetBase.StatefulWidgetBase object …

## Todo: Export Seed

In [9]:
explorer.state.data.datatable[0]['highlight']

AttributeError: 'ExplorerESE' object has no attribute 'state'

In [19]:
prov = explorer.history()
prov.show()

component(7f049477f460, self=<kyurem.widgets.StatefulWidgetBase.StatefulWidgetBase object at 0x7f049792ac70>)

LayoutWidget(Layout(component(7f049477f460, self=<kyurem.widgets.StatefulWidgetBase.StatefulWidgetBase object …

# Merging Decision Making

## Load Merge Data

In [20]:
df2 = pd.read_csv("indeed.v0.0.1_612e3403_merged.csv")
merge_data = df2.to_dict('records')

service.load_merge_data(merge_data, 'corpus_entity', 'node_label', 'node_uuid', 'node_title')
mergedata = service.get_merge_data()["rows"]

In [21]:
decision_list = ["Accept", "Reject", "Defer"]

## Verify Merging Recommendations

In [24]:
from kyurem import MergeVerifier


def init(state): 
    # Fetch initial view data
    return {
        "subgraph": None,
        "corpus": None,
        "mergedata":mergedata,
        "decisions":decision_list,
    }

def focus(state, row, panel):
    # node can be in the following format:
    # node : { "node_label": str, "node_property": str, "node_property_value": str }
    node = { "node_label": row["node_label"], "node_property": "title", "node_property_value": row["node"] }
    entity = { "node_label": row["node_label"], "node_property": "title", "node_property_value": row["entity"] }
    # Fetch the neighborhood around the input node 
    result = service.get_node_neighborhood(node)
    datatable = service.get_annotated_corpus(entity)
    
    data = {}
    data["subgraph"] = result["schema"]
    # TODO: Move conversion code into service
    if datatable["rows"]:
        data["corpus"] = datatable["rows"]
        data["highlight"] = datatable["highlight"]
    else:
        data["corpus"] = None
        data["highlight"] = None
    return data
  
# Create widget 
reload_bundle()
merge_verifier = MergeVerifier({
        "init": init,
        "focus": focus
    }, mergedata)

merge_verifier.show()

kyurem_client: 0.0.0


[autoreload of kyurem failed: Traceback (most recent call last):
  File "/home/fchoi/.local/share/virtualenvs/kyurem-migration-FfkIKKAu/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/home/fchoi/.local/share/virtualenvs/kyurem-migration-FfkIKKAu/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    module = reload(module)
  File "/home/fchoi/.pyenv/versions/3.9.13/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/home/fchoi/git/kyurem-migration/src/kyurem/__init__.py", line 35, in <module>
    from .widgets.MergeVerifier import MergeVerifier
  File "/home/fchoi/git/kyurem-migration/src/kyurem/widgets/Me

ImportError: cannot import name 'MergeVerifier' from 'kyurem' (/home/fchoi/git/kyurem-migration/src/kyurem/__init__.py)

## Todo: Export Merge Data

In [ ]:
merge_verifier.state.data.mergedata